In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

from model import LeNet
from dataloader import MNISTDataLoader
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms

In [11]:
torch.backends.cudnn.enabled = False
torch.manual_seed(1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def show_images(dataloader, num_images):
    for batch_idx, (data, target) in enumerate(dataloader):
        for i in range(len(data)):
            plt.imshow(data[i].squeeze(), cmap='gray')
            plt.title(f"Label: {target[i]}")
            plt.axis('off')
            plt.show()

            num_images -= 1
            if num_images == 0:
                return

class NoReweighting():
    def __init__(self, network, hyperparameters, criterion, optimizer, train_loader, test_loader):
        self.network = network
        self.hyperparameters = hyperparameters
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.test_loader = test_loader

    def train(self):
        # Train the network
        for epoch in range(hyperparameters['n_epochs']):
            self.network.train()
            for batch_idx, (data, target) in enumerate(self.train_loader):
                data = data.to(device)
                target = target.to(device)

                self.optimizer.zero_grad()
                output = self.network(data)

                loss = self.criterion(output, target.float())
                loss.backward()
                self.optimizer.step()
                if batch_idx % hyperparameters['log_interval'] == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), loss.item()))


    def test(self):
        test_loss = 0
        correct = 0
        total_samples = 0

        self.network.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(self.test_loader):
                output = self.network(data.to(device)).cpu()
                batch_size = data.size(0)
                total_samples += batch_size
                test_loss += self.criterion(output, target.float()).item()
                pred = (torch.sigmoid(output) > 0.5).int()
                correct += (pred == target.int()).sum().item()

        return correct / total_samples 

hyperparameters = {
    'n_epochs' : 10,
    'batch_size_train' : 100,
    'batch_size_valid' : 10,
    'batch_size_test' : 1000,
    'learning_rate' : 1e-3,
    'momentum' : 0.5,
    'log_interval' : 50
}

avging_size = 5
perc_9_arr = [100, 25, 10, 5, 1, 0.5]

df = pd.DataFrame(columns=[str(x) for x in perc_9_arr])

# for perc in perc_9_arr:
#     acc_arr = []
#     for repeat in range(avging_size):
network = LeNet()

criterion = nn.BCEWithLogitsLoss(reduction='none')
criterion_mean = nn.BCEWithLogitsLoss(reduction='mean')

optimizer = optim.SGD(network.params(),
                        lr=hyperparameters['learning_rate'],
                        momentum=hyperparameters['momentum'])
# # Load the data
# data_loader = MNISTDataLoader(validation_ratio=0.05,
#                             batch_size_train=hyperparameters['batch_size_train'],
#                             batch_size_valid=hyperparameters['batch_size_valid'],
#                             batch_size_test=hyperparameters['batch_size_test'])

# desired_sample_distribution = [100, perc]
# data_loader.sample_bias(desired_sample_distribution, dataset="train")

# train_loader = data_loader.train_dataloader
# valid_loader = data_loader.valid_dataloader
# test_loader = data_loader.test_dataloader

train_folder = './dataset-ninja/train_binary'
test_folder = './dataset-ninja/test_binary'
validate_folder = './dataset-ninja/validate_binary'

class_weights = [0.5, 0.5]  # Example weights for each class

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=(1/2.803307641932519,), std=(0.16677909497124954,)),  # Normalize images
])

train_dataset = datasets.ImageFolder(train_folder, transform=transform)
weights = [class_weights[label] for label in train_dataset.targets]
train_sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

test_dataset = datasets.ImageFolder(test_folder, transform=transform)

validate_dataset = datasets.ImageFolder(validate_folder, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size_train'], sampler=train_sampler)
# train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size_test'], shuffle=True)
valid_loader = DataLoader(validate_dataset, batch_size=hyperparameters['batch_size_valid'], shuffle=True)

# show_images(test_loader, 100)

our_model = NoReweighting(network, hyperparameters, criterion_mean, optimizer, train_loader, test_loader)

# Starting accuracy
accuracy = our_model.test()
print("Starting accuracy: ", accuracy)

# Roughly 1 min per loop
for lcv in range(30):
    our_model.train()

    # Ending accuracy
    accuracy = our_model.test()
    print("Current accuracy: ", accuracy)
#         acc_arr.append(accuracy)
#         # print("testing " + str(perc) + " accuracy = " + str(accuracy))
#     df[str(perc)] = acc_arr
#     print(df)
# print(df)
# df.to_csv("accuracy_tsting_no_weights.csv")


Starting accuracy:  0.5
Train Epoch: 0 [0/6099 (0%)]	Loss: 0.695603
Train Epoch: 0 [5000/6099 (82%)]	Loss: 0.693040
Train Epoch: 1 [0/6099 (0%)]	Loss: 0.696429
Train Epoch: 1 [5000/6099 (82%)]	Loss: 0.697039
Train Epoch: 2 [0/6099 (0%)]	Loss: 0.694451
Train Epoch: 2 [5000/6099 (82%)]	Loss: 0.691264
Train Epoch: 3 [0/6099 (0%)]	Loss: 0.690961
Train Epoch: 3 [5000/6099 (82%)]	Loss: 0.693016
Train Epoch: 4 [0/6099 (0%)]	Loss: 0.697455
Train Epoch: 4 [5000/6099 (82%)]	Loss: 0.692690
Train Epoch: 5 [0/6099 (0%)]	Loss: 0.691792
Train Epoch: 5 [5000/6099 (82%)]	Loss: 0.692201
Train Epoch: 6 [0/6099 (0%)]	Loss: 0.693731
Train Epoch: 6 [5000/6099 (82%)]	Loss: 0.695687
Train Epoch: 7 [0/6099 (0%)]	Loss: 0.691357
Train Epoch: 7 [5000/6099 (82%)]	Loss: 0.694185
Train Epoch: 8 [0/6099 (0%)]	Loss: 0.691136
Train Epoch: 8 [5000/6099 (82%)]	Loss: 0.691292
Train Epoch: 9 [0/6099 (0%)]	Loss: 0.690877
Train Epoch: 9 [5000/6099 (82%)]	Loss: 0.690856
Current accuracy:  0.59
Train Epoch: 0 [0/6099 (0%)]	Los

In [3]:
# Testing MNIST dataset
hyperparameters = {
    'n_epochs' : 10,
    'batch_size_train' : 100,
    'batch_size_valid' : 10,
    'batch_size_test' : 1000,
    'learning_rate' : 1e-3,
    'momentum' : 0.5,
    'log_interval' : 50
}

data_loader = MNISTDataLoader(validation_ratio=0.05,
                            batch_size_train=hyperparameters['batch_size_train'],
                            batch_size_valid=hyperparameters['batch_size_valid'],
                            batch_size_test=hyperparameters['batch_size_test'])

desired_sample_distribution = [100, 100]
data_loader.sample_bias(desired_sample_distribution, dataset="train")

train_loader = data_loader.train_dataloader

for batch_idx, (data, target) in enumerate(train_loader):
    print

print(data[0, 0, :, :])

print(torch.mean(data[0, 0, :, :]))
print(torch.std(torch.flatten(data[0, 0, :, :])))

tensor([[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
        [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
        [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
        ...,
        [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
        [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
        [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]])
tensor(0.0080)
tensor(0.9482)


In [9]:
# Testing Car dataset
train_folder = './dataset-ninja/train_binary'
test_folder = './dataset-ninja/test_binary'
validate_folder = './dataset-ninja/validate_binary'

class_weights = [0.5, 0.5]  # Example weights for each class

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=(1/2.803307641932519,), std=(0.16677909497124954,)),  # Normalize images
])

train_dataset = datasets.ImageFolder(train_folder, transform=transform)
weights = [class_weights[label] for label in train_dataset.targets]
train_sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

data_vec = np.array([])
for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
    data_vec = np.append(data_vec, torch.flatten(data[0, 0, :, :]).numpy())
    if batch_idx > 2000:
        break

print(data[0, 0, :, :])

print(np.mean(data_vec))
print(np.std(data_vec))

2001it [00:10, 183.93it/s]

tensor([[ 2.0465,  2.4463,  3.7160,  ...,  1.4587,  0.8944,  0.8238],
        [ 2.1406,  2.5638,  3.8100,  ...,  1.2471,  0.6357, -0.1167],
        [ 1.9054,  2.4698,  3.7630,  ...,  3.7630,  3.0106,  1.1295],
        ...,
        [-0.3048,  0.2125, -0.8456,  ...,  1.1295,  1.1060,  0.7768],
        [ 0.1654, -0.5165, -0.8221,  ...,  3.1281,  3.0576,  2.6344],
        [ 0.4946, -0.0697, -0.1637,  ...,  3.4809,  3.4573,  3.3868]])
0.010853047370362943
1.017077427591781
